In [1]:

import sys
import pandas as pd
sys.path.append(r'..\src')
import numpy as np
from MGfold import mgfold, mg_dot_bracket, mg_s_matrix
from tqdm import tqdm
import time

df = pd.read_csv(r'..\data\published_clean_files\merged_clean_data.csv')

In [2]:

percentile_95 = np.percentile(df['Sequence'].apply(len), 95) 
percentile_5 = np.percentile(df['Sequence'].apply(len), 5)
df = df[df['Sequence'].apply(len).between(np.floor(percentile_5)+1, np.ceil(percentile_95))]
df = df.reset_index(drop=True)
print(len(df))
lengths = df['Sequence'].apply(len)
max_length = lengths.max()
print('max length:', lengths.max())
print('min length:', lengths.min())

4450
max length: 83
min length: 33


In [3]:
d_y= { 'Energy': [], 'time': [], 'Sequence': [] , 'Count': [], 'File':[], 'd_b': [], 'faces': [], 'energy_faces': [], 's_matrix':[]}
for count, seq in tqdm(enumerate(df['Sequence'])):
        start_time = time.time()
        structs = mgfold(seq, l_fix =4)
        d_y['time'].append(time.time()-start_time)
        sss = 0
        for s in structs:     
            sss =s.e
        d_b = mg_dot_bracket( seq, structs)
        d_y['faces'].append([s.desc for s in structs])
        d_y['energy_faces'].append([s.e for s in structs])
        d_y['Energy'].append(sss)
        d_y['Sequence'].append(seq)
        d_y['Count'].append(df['Count'][count])
        d_y['File'].append(df['File'][count])
        d_y['d_b'].append(d_b)
        d_y['s_matrix'].append(mg_s_matrix(seq, structs, max_length).tolist())
df2 = pd.DataFrame(d_y)
df2.to_csv(f'test_fold_published.csv', index=False)
df2.head()


0it [00:00, ?it/s]

4450it [02:33, 28.91it/s]


,Energy,time,Sequence,Count,File,d_b,faces,energy_faces,s_matrix
0,1.5,0.026337,ACGACGGGGCACATTGTGCTGTTCATCTGTTCCGCAGGAGAGTCGT,77352,N48 after 13th,(((((...((((...))))............((...))...))))),"[STACK:AC/TG, STACK:CG/GC, STACK:GA/CT, STACK:...","[-1.5, -2.2, -1.3, -1.5, 5.3, -2.2, -1.5, -1.5...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,4.6,0.020776,ACGACCACCTAGGTATTCATGACCGTCTAGAGCTTTCATTGGTCGT,67049,N58 after 16th,(((((((...........((((.............))))))))))),"[STACK:AC/TG, STACK:CG/GC, STACK:GA/CT, STACK:...","[-1.5, -2.2, -1.3, -1.5, -1.8, -1.5, 4.9, -0.9...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,1.5,0.018127,ACGACGGGGCACATTGTGCTATTCAGTTGTTCCGCAGGAGAGTCGT,48708,N58 after 12th,(((((...((((...))))............((...))...))))),"[STACK:AC/TG, STACK:CG/GC, STACK:GA/CT, STACK:...","[-1.5, -2.2, -1.3, -1.5, 5.3, -2.2, -1.5, -1.5...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,1.5,0.012213,ACGACGGGGCACATTGTGCTGTCCATCTGTTCCGCAGGAGAGTCGT,28432,N58 after 16th,(((((...((((...))))............((...))...))))),"[STACK:AC/TG, STACK:CG/GC, STACK:GA/CT, STACK:...","[-1.5, -2.2, -1.3, -1.5, 5.3, -2.2, -1.5, -1.5...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,2.4,0.016466,ACGACGGGGCACATTGTGCTGTTCACCTGTTCCGCAGGAGAGTCGT,12126,N48 after 9th,(((((...((((...))))...((.((((.....)))).))))))),"[STACK:AC/TG, STACK:CG/GC, STACK:GA/CT, STACK:...","[-1.5, -2.2, -1.3, -1.5, 4.1, -2.2, -1.5, -1.5...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [5]:
#check that the sequences, associated secondary structures and per face3 energies in the generated file are the same as the one used for the experiments
# in the file fold_published.csv. This check is as order of the sequences within the files may be different
df1 = pd.read_csv(r'../data/fold_published.csv')
df2 = pd.read_csv(r'.\test_fold_published.csv')
error = []
error1 =[]
error2 = []
for i in range(len(df1)):
    if (df1['d_b'][i] != df2['d_b'].loc[df2['Sequence']== df1['Sequence'][i]]).values[0]:
        error.append(i)
    if (df1['faces'][i] != df2['faces'].loc[df2['Sequence']== df1['Sequence'][i]]).values[0]:
        error1.append(i)
    if (df1['energy_faces'][i] != df2['energy_faces'].loc[df2['Sequence']== df1['Sequence'][i]]).values[0]:
        error2.append(i)
print(error, error1,  error2 )

[] [] []
